In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gps-data/data.csv
/kaggle/input/gps-dataset/blackmesa.geojson
/kaggle/input/gps-dataset/Redv1.geojson
/kaggle/input/gps-dataset/Redv2.geojson


In [2]:
!pip install movingpandas

  Obtaining dependency information for movingpandas from https://files.pythonhosted.org/packages/d4/ad/9fd37601f8766cab5dac68685f5e472fdc7c89fc409a303fa1ab513baa7e/movingpandas-0.17.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install hvplot

  Obtaining dependency information for hvplot from https://files.pythonhosted.org/packages/a9/fc/f2daa47260663cec3dd0a4c9f0592f27591d84d77f1178662010edb97aa6/hvplot-0.9.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}

mpd.show_versions()


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
executable : /opt/conda/bin/python
machine    : Linux-5.15.133+-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.11.1
GEOS lib   : /opt/conda/lib/libgeos_c.so
GDAL       : 3.6.4
GDAL data dir: /opt/conda/lib/python3.10/site-packages/fiona/gdal_data
PROJ       : 9.3.1
PROJ data dir: /opt/conda/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.1
pandas     : 2.0.3
fiona      : 1.9.5
numpy      : 1.24.3
shapely    : 1.8.5.post1
rtree      : 1.1.0
pyproj     : 3.6.1
matplotlib : 3.7.4
mapclassify: 2.6.0
geopy      : 2.4.1
holoviews  : 1.18.1
hvplot     : 0.9.2
geoviews   : 1.11.0
stonesoup  : None


In [5]:
df = gpd.read_file('/kaggle/input/gps-dataset/blackmesa.geojson')

In [6]:
df

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,DOP,Satellites,Cause of F,Lab_Coord,...,Z_mean,Z_std,ODBA_mean,ODBA_std,VeDBA_mean,VeDBA_std,FMR_4fac,FMG_4fac,FMT_4fac,geometry
0,2022-10-15 15:20:00,32.453676,-105.864855,1990.94,31,15.0,0.8,10,GPS Schedule,0.0,...,5.491329,17.776590,43.254335,10.924282,34.332384,5.297903,1.001000,0.002559,0.001000,POINT (982914.328 3602356.126)
1,2022-10-15 15:30:00,32.449636,-105.868142,1989.17,20,15.5,1.0,7,GPS Schedule,0.0,...,-8.140704,17.114462,44.562814,10.739717,32.144722,5.774948,1.000000,0.001882,0.000000,POINT (982626.456 3601892.568)
2,2022-10-15 15:40:00,32.443670,-105.875221,1990.33,14,18.0,2.4,6,GPS Schedule,0.0,...,0.798995,16.191374,49.386935,8.314547,32.463978,5.126508,1.000000,0.003336,0.000000,POINT (981991.878 3601197.988)
3,2022-10-15 15:50:00,32.440500,-105.857006,1989.91,14,20.0,1.0,9,GPS Schedule,0.0,...,-16.361809,20.464696,35.768844,10.584279,29.342299,4.524011,1.000000,0.000295,0.000000,POINT (983723.913 3600928.630)
4,2022-10-15 16:00:00,32.455261,-105.864170,2004.84,16,21.5,1.2,8,GPS Schedule,0.0,...,-13.487437,7.408307,47.296482,9.250698,30.070202,4.396666,0.000000,0.000000,0.300000,POINT (982970.379 3602535.194)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9315,2023-01-31 17:20:00,33.994083,-105.215451,1698.26,12,9.5,2.8,5,GPS Schedule,0.0,...,6.366834,0.492257,37.552764,0.654309,31.809075,0.384163,0.945132,0.000479,0.016460,POINT (1034569.956 3776618.326)
9316,2023-01-31 17:30:00,33.993316,-105.214653,1658.09,19,9.5,3.0,4,GPS Schedule,0.0,...,6.376884,0.543272,37.587940,0.796383,31.813417,0.397171,0.968611,0.000495,0.009417,POINT (1034648.668 3776537.327)
9317,2023-01-31 17:40:00,33.990100,-105.210642,1662.54,34,10.5,1.6,5,GPS Schedule,0.0,...,6.336683,0.710447,37.532663,0.981071,31.806311,0.559768,0.986331,0.000497,0.004101,POINT (1035040.068 3776201.112)
9318,2023-01-31 17:50:00,33.996950,-105.223902,1635.13,13,10.5,1.6,5,GPS Schedule,0.0,...,6.276382,1.997233,39.175879,3.876773,32.084948,2.564411,0.000000,0.000000,0.300000,POINT (1033769.965 3776892.618)


In [7]:
df['t'] = pd.to_datetime(df['GMT Time'])
df = df.set_index('t').tz_localize(None)

In [8]:
df.head()

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,DOP,Satellites,Cause of F,Lab_Coord,...,Z_mean,Z_std,ODBA_mean,ODBA_std,VeDBA_mean,VeDBA_std,FMR_4fac,FMG_4fac,FMT_4fac,geometry
t,,,,,,,,,,,,,,,,,,,,,
2022-10-15 15:20:00,2022-10-15 15:20:00,32.453676,-105.864855,1990.94,31,15.0,0.8,10,GPS Schedule,0.0,...,5.491329,17.776590,43.254335,10.924282,34.332384,5.297903,1.001,0.002559,0.001,POINT (982914.328 3602356.126)
2022-10-15 15:30:00,2022-10-15 15:30:00,32.449636,-105.868142,1989.17,20,15.5,1.0,7,GPS Schedule,0.0,...,-8.140704,17.114462,44.562814,10.739717,32.144722,5.774948,1.000,0.001882,0.000,POINT (982626.456 3601892.568)
2022-10-15 15:40:00,2022-10-15 15:40:00,32.443670,-105.875221,1990.33,14,18.0,2.4,6,GPS Schedule,0.0,...,0.798995,16.191374,49.386935,8.314547,32.463978,5.126508,1.000,0.003336,0.000,POINT (981991.878 3601197.988)
2022-10-15 15:50:00,2022-10-15 15:50:00,32.440500,-105.857006,1989.91,14,20.0,1.0,9,GPS Schedule,0.0,...,-16.361809,20.464696,35.768844,10.584279,29.342299,4.524011,1.000,0.000295,0.000,POINT (983723.913 3600928.630)
2022-10-15 16:00:00,2022-10-15 16:00:00,32.455261,-105.864170,2004.84,16,21.5,1.2,8,GPS Schedule,0.0,...,-13.487437,7.408307,47.296482,9.250698,30.070202,4.396666,0.000,0.000000,0.300,POINT (982970.379 3602535.194)


In [9]:
original_crs = df.crs
original_crs

<Projected CRS: EPSG:26912>
Name: NAD83 / UTM zone 12N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 114°W and 108°W - onshore and offshore. Canada - Alberta; Northwest Territories; Nunavut; Saskatchewan. United States (USA) - Arizona; Colorado; Idaho; Montana; New Mexico; Utah; Wyoming.
- bounds: (-114.0, 31.33, -108.0, 84.0)
Coordinate Operation:
- name: UTM zone 12N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
df.to_crs({'init': 'epsg:4326'}).hvplot(title='Geographic extent of the dataset', geo=True, tiles='OSM')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [11]:
df = df.to_crs({'init': 'epsg:4326'})
( df.hvplot(title='OSM showing paths and fences', size=2, geo=True, tiles='OSM') +
  df.hvplot(title='Imagery showing land cover details', size=2, color='red', geo=True, tiles='EsriImagery') )

:Layout
   .Overlay.I  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [12]:
from pyproj import CRS
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim, Layout
from os.path import exists
from urllib.request import urlretrieve

In [13]:
temp = df.to_crs(CRS(25832))
temp['geometry'] = temp['geometry'].buffer(5)
total_area = temp.dissolve(by='No').area 


In [14]:
print("The dataset covers the time between {} and {}.".format(df.index.min(), df.index.max()))

The dataset covers the time between 2022-10-15 15:20:00 and 2023-01-31 18:00:00.


In [15]:
print("That's {}".format(df.index.max() - df.index.min()))

That's 108 days 02:40:00


In [16]:
df['No'].resample('1d').count().hvplot(title='Number of records per day')

:Curve   [t]   (No)

In [17]:
df['Y-M'] = df.index.to_period('M')    
Layout([df[df['Y-M']==i].hvplot(title=str(i), size=2, geo=True, tiles='OSM') for i in df['Y-M'].unique()])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [18]:
t = df.reset_index().t
df = df.assign(delta_t=t.diff().values)
df['delta_t'] = df['delta_t'].dt.total_seconds()/60
pd.DataFrame(df).hvplot.hist('delta_t', title='Histogram of intervals between consecutive records (in minutes)', bins=60, bin_range=(0, 60))

:Histogram   [delta_t]   (delta_t_count)

In [19]:
df['CollarId']=2

In [20]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]
# traj.add_speed()
# max_speed = traj.df.speed.max()
# print("The highest computed speed is {:,.2f} m/s ({:,.2f} km/h)".format(max_speed, max_speed*3600/1000))

In [21]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]

In [22]:
pd.DataFrame(traj.df).hvplot.hist('speed', title='Histogram of speeds (in meters per minute)', bins=90)

:Histogram   [speed]   (speed_count)

In [23]:
traj.add_direction(overwrite=True)
pd.DataFrame(traj.df).hvplot.hist('direction', title='Histogram of directions', bins=90)

:Histogram   [direction]   (direction_count)

In [24]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by hour of day and month of year', 
                                     x='t.hour', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [t.hour,t.month]   (speed)

In [25]:
traj.df['n'] = 2
pd.DataFrame(traj.df).hvplot.heatmap(title='Record count by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='n', reduce_function=np.sum)

:HeatMap   [Temperatur,t.month]   (n)

In [26]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [Temperatur,t.month]   (speed)

In [27]:
traj.df['dir_class'] = ((traj.df['direction']-22.5)/45).round(0)
temp = traj.df
Layout([temp[temp['dir_class']==i].hvplot(geo=True, tiles='OSM', size=2, width=300, height=300, title=str(int(i*45))+"°") for i in sorted(temp['dir_class'].unique())])

:Layout
   .Overlay.I    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [28]:
traj.df['speed_class'] = (traj.df['speed']*2).round(1)
temp = traj.df
plots = []
for i in sorted(temp['speed_class'].unique()):
    filtered = temp[temp['speed_class']==i]
    if len(filtered) < 10: 
        continue
    plots.append(filtered.hvplot(geo=True, tiles='EsriImagery', color='red', size=2, width=300, height=300, title=str(i/2)))
Layout(plots)

:Layout
   .Overlay.I        :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V        :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IX       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.X        :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XI       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XII      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIII     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIV      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XV       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVI      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVII     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVIII    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIX      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XX       :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXI      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXII     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXIII    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXIV     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXV      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXVI     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXVII    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXVIII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXIX     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXX      :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXXI     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXXII    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XXXIII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points

In [29]:
daily = mpd.TemporalSplitter(tc).split(mode='day')

In [30]:
Layout([daily.trajectories[i].hvplot(title=daily.trajectories[i].id, c='speed', line_width=2, cmap='RdYlBu') for i in range(0,7)])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.IV  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.V   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VI  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

In [31]:
daily_starts = daily.get_start_locations()
daily_starts.set_index(pd.to_datetime(daily_starts['GMT Time']), inplace=True) 
daily_starts['month'] = daily_starts.index.month
daily_starts.hvplot(c='month', geo=True, tiles='EsriImagery', cmap='autumn')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (month)

In [32]:
moving = mpd.TrajectoryCollection(traj.df[traj.df['speed'] > 0.05], 'CollarId')
moving = mpd.ObservationGapSplitter(moving).split(gap=timedelta(minutes=70))

In [33]:
moving.get_start_locations().hvplot(c='month', geo=True, tiles='EsriImagery', color='red')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [34]:
daily_lengths = [traj.get_length() for traj in daily]
daily_t = [traj.get_start_time() for traj in daily]

In [35]:
daily_lengths = pd.DataFrame(daily_lengths, index=daily_t, columns=['length'])
daily_lengths.hvplot(title='Daily trajectory length')

:Curve   [index]   (length)